In [ ]:

import pandas as pd
import numpy as np
import os, logging, traceback, sys

from datetime import datetime

logger = logging.getLogger('o9_logger')

class MasterData:

    logger.info("---------------MDM Commit START-----------------------------")
    
    def __init__(self):
        # initialize all variables in init function
        logger.info("---------------Initial Data-----------------------------")
            
        #user_input from Python
        #self.user_input = pd.read_csv('Fact.Download_userinput.csv')
        self.user_input = user_input
        logger.info('{}'.format(self.user_input.to_string()))
        #self.user_input = self.user_input.replace(r'^\s*$', np.nan, regex=True)
        #self.user_input = self.user_input.where(pd.notnull(self.user_input), None)
        
        #self.user_input.fillna('', inplace=True)
        self.user_input_out = self.user_input.copy()
        logger.info('{}'.format(self.user_input_out.to_string()))        

        #master_data from Python
        #self.dim_data = pd.read_csv('Dimension.Download_dimdata.csv')
        self.dim_data = master_data
        logger.info('{}'.format(self.dim_data.to_string()))
    
        #mapping_data from Python
        #self.mapping = pd.read_csv('Fact.Download_mapping_data.csv')
        self.mapping = mapping_data
        logger.info('{}'.format(self.mapping.to_string()))
    
        #hierarchy_data
        #self.hierarchy = pd.read_csv('Dimension.Download_hierarchydata.csv')
        self.hierarchy = hierarchy_data
        logger.info('{}'.format(self.hierarchy.to_string()))
            
        self.out_MG = pd.DataFrame()
    
        self.Total_attributes = len(self.mapping.index)
        
        #self.leafattributename = 'Location.[Location]'
        self.leafattributename = leafattributename
        
        #### Updated on 18th feb 2022 start
        ls_dimensionname = self.leafattributename.split('.')
        #### Updated on 18th feb 2022 end
        
        logger.info(self.leafattributename)
        logger.info(ls_dimensionname)
        
        #self.username = 'girish'
        self.username = usernamevalue
        logger.info(self.username)
        
        #self.transaction_id = 5
        self.transaction_id = transactionidvalue
        logger.info(self.transaction_id)
        
        self.current_time = datetime.now()
        logger.info(self.current_time)
        
        #self.keyattributename = 'Location.[LocationKey]'
        self.keyattributename = keyattributename
        logger.info(self.keyattributename)
        self.List = ['List'] * (self.user_input_out.shape[0])
        #self.List = ['List'] * 1000000
        logger.info(self.List)
        
        #### Updated on 18th feb 2022 start
        self.hierarchy = self.hierarchy[self.hierarchy['_SchemaDimension.[_Dimension]'] == ls_dimensionname[0]]
        self.hierarchy = self.hierarchy.reset_index(drop=True)
        #self.hierarchy.to_csv('hierarchy_1.csv')
        logger.info('{}'.format(self.hierarchy.to_string()))
        
        #### Updated on 18th feb 2022 end
             
    def Rename_Column(self):
        logger.info("---------------Rename Column Start-----------------------------")
        for l in range(self.Total_attributes):
            #index_no = self.mapping.columns.get_loc('Is Mandatory')
            Mandatory = self.mapping['Is Mandatory'].values[l]
            logger.info(Mandatory)
            Atribute_Name = self.mapping['Mapping Attribute'].values[l]
            logger.info(Atribute_Name)
            Atribute_Name = 'IN ' + Atribute_Name
            if(Mandatory ==True):
                logger.info(Atribute_Name)
                self.user_input[Atribute_Name] = self.user_input[Atribute_Name].replace(r'^\s*$', np.nan, regex=True)
                self.user_input[Atribute_Name] = self.user_input[Atribute_Name].where(pd.notnull(self.user_input[Atribute_Name]), None)
            elif(Mandatory ==False):
                logger.info(Atribute_Name)
                self.user_input[Atribute_Name] = self.user_input[Atribute_Name].replace(r'^\s*$', 'No Value Provided', regex=True)
                self.user_input[Atribute_Name] = self.user_input[Atribute_Name].replace(np.nan, 'No Value Provided', regex=True)
        logger.info('{}'.format(self.user_input_out.to_string()))
        for i in range(self.Total_attributes):
            #logger.info(i)
            index_no = self.mapping.columns.get_loc('Dim Attribute.[Dim Attribute Name]')
            #logger.info(index_no)
            oldName = self.mapping.iloc[i, index_no + 1]
            #logger.info(oldName)
            
            #### Updated on 18th feb 2022 start
            oldName = 'IN ' + oldName
            #print(oldName)
            logger.info(oldName)
            #### Updated on 18th feb 2022 end
            #logger.info(oldName)
            
            newName = self.mapping.iloc[i, index_no]
            logger.info(newName)
            self.user_input.rename(columns={oldName: newName}, inplace=True)
                
        logger.info('{}'.format(self.user_input.to_string()))
        logger.info("---------------Rename Column End-----------------------------")
    
    def Netchange(self):
        logger.info("---------------Net Change Start-----------------------------")
        logger.info("---------------Data to this Function-----------------------------")
        logger.info('{}'.format(self.dim_data.to_string()))
        logger.info('{}'.format(self.user_input.to_string()))
        logger.info(self.keyattributename)
        #first_column = self.dim_data.columns[-1]
        first_column = self.keyattributename
        #logger.info(first_column)
        logger.info(first_column) 
        # To Get leaf Attribute key for that dimension
        # To Change the datatype from Float to Object
        #SD#
        self.dim_data[first_column] = self.dim_data[first_column].astype(int).astype(str)
        #self.dim_data = self.dim_data.astype(object)
        #self.user_input = self.user_input.astype(object)
        logger.info('{}'.format(self.dim_data.to_string()))
        logger.info('{}'.format(self.user_input.to_string()))

        #To rename the LS key column to Unique Identifier.[Unique Identifier]
        self.dim_data.rename(columns={first_column: 'Unique Identifier.[Unique Identifier]'}, inplace=True)
        self.dim_data = self.dim_data.astype(object)
        self.user_input = self.user_input.astype(object)
        #logger.info('{}'.format(self.dim_data.to_string()))

        self.out_MG = self.user_input.merge(self.dim_data, how='outer', indicator=True).loc[
            lambda x: x['_merge'] == 'left_only']
        logger.info('{}'.format(self.out_MG.to_string()))
        
        test_df = self.user_input.merge(self.dim_data, how='inner', indicator=False)
        column_count1 = len(test_df.columns)
        test_df = test_df[test_df[test_df.columns[column_count1 - 6]] == 'Error']
        logger.info('{}'.format(test_df.to_string()))
        
        del self.out_MG['_merge']
        frames = [self.out_MG,test_df]
        self.out_MG = pd.concat(frames)
        #logger.info('{}'.format(test_df2.to_string()))
        self.out_MG.columns = self.user_input_out.columns
        self.out_MG.columns = self.out_MG.columns.str.replace("IN", "OUT")
        column_count = len(self.out_MG.columns)
        self.out_MG[self.out_MG.columns[column_count - 6]] = 'Modified'
        self.out_MG[self.out_MG.columns[column_count - 5]] = self.username
        self.out_MG[self.out_MG.columns[column_count - 4]] = self.current_time.strftime("%Y-%m-%d %H:%M:%S")
        self.out_MG[self.out_MG.columns[column_count - 3]] = np.nan
        filter = self.out_MG['Unique Identifier.[Unique Identifier]']
        
        for x in filter:
            if x not in self.dim_data['Unique Identifier.[Unique Identifier]'].values:
                mask = (self.out_MG['Unique Identifier.[Unique Identifier]'] == x)
                self.out_MG[self.out_MG.columns[column_count - 6]][mask] = 'Added'
            
        logger.info("---------------Result Set of Netchange-----------------------------")
        self.out_MG.reset_index(inplace = True, drop = True)
        logger.info('{}'.format(self.out_MG.to_string()))
        logger.info("---------------Net Change End-----------------------------")
    
    def Mandatory_datacheck(self):
        logger.info("---------------Mandatory Start-----------------------------")
        logger.info("---------------Data to this Function-----------------------------")
        logger.info('{}'.format(self.out_MG.to_string()))
        logger.info('{}'.format(self.mapping.to_string()))
        logger.info('{}'.format(self.user_input.to_string()))
        rows = self.out_MG.count
        for k in range(self.Total_attributes):
            #Mandatory = self.mapping.iloc[k, 4]
            Attribute = self.out_MG.columns[k + 3]
            logger.info(Attribute)
            Attribute1 = self.user_input.columns[k + 3]
            logger.info(Attribute1)
            Mandatory = self.mapping.loc[self.mapping['Dim Attribute.[Dim Attribute Name]'] == Attribute1, 'Is Mandatory'].item()
            logger.info(Mandatory)
            column_count = len(self.out_MG.columns)
                    
            if (Mandatory):
                if self.out_MG[Attribute].isnull().any():
                    Missing_ReportLabel = self.mapping.loc[self.mapping['Dim Attribute.[Dim Attribute Name]'] == Attribute1, 'Report Label'].item()
                    index_list = self.out_MG[self.out_MG[Attribute].isnull()].index.tolist()
                    for i in index_list:
                        if (self.List[i] == 'List'):
                            self.List[i] = self.List[i] + 'Data missing for mandatory attribute ' + Missing_ReportLabel
                        else:
                            self.List[i] = self.List[i] +', '+ Missing_ReportLabel
                    mask = self.out_MG[Attribute].isnull()
                    self.out_MG[self.out_MG.columns[column_count - 6]][mask] = 'Error'
            
        logger.info("---------------Result Set of MandatoryCheck-----------------------------")
        logger.info('{}'.format(self.out_MG.to_string()))
        logger.info("--------------Mandatory End-----------------------------")
        
    def Data_Typecheck(self):
            logger.info("---------------Data_Typecheck Start-----------------------------")
            for y in range(self.Total_attributes):
    
                data_type = self.mapping['Data Type'].values[y]
                logger.info(data_type)
                Attribute = self.mapping['Mapping Attribute'].values[y]
                Mandatory = self.mapping['Is Mandatory'].values[y]
                Attribute = 'OUT ' + Attribute
                reportlabel_leafattributename = self.mapping['Report Label'].values[y]
                column_count = len(self.out_MG.columns)
    
                logger.info(Attribute)
    
                if(data_type == 'string'):
                    continue
                
                elif(data_type == 'number'):
                    df_datacheck3 = self.out_MG.copy()
                    df_datacheck3['check'] = df_datacheck3[Attribute]
                    #df_datacheck3['Is_Numeric'] = list(map(lambda x: x.isnumeric(), df_datacheck3[Attribute]))
                    #df_datacheck3 = df_datacheck3[Attribute].apply(lambda x: x.str.isnumeric())
                    #df_datacheck3['Is_Numeric'] = df_datacheck3[Attribute].apply(pd.to_numeric, errors="ignore").applymap(lambda x: isinstance(x, float), na_action='ignore')
                    df_datacheck3[Attribute] = pd.to_numeric(df_datacheck3[Attribute], errors='coerce')
                    df_datacheck3[Attribute] = df_datacheck3[Attribute].replace(np.nan,'False',regex = True)
                    logger.info('{}'.format(df_datacheck3.to_string()))
                    index_list = df_datacheck3.index[df_datacheck3[Attribute] == 'False'].tolist()
                    for i in index_list:
                        if(df_datacheck3['check'].values[i] != None and Mandatory):
                            self.out_MG[self.out_MG.columns[column_count - 6]].values[i] = 'Error'
                            self.List[i] = self.List[i] +'The ' +reportlabel_leafattributename+ ' expected datatype is '+ data_type + ', '
                        elif(df_datacheck3['check'].values[i] != 'No Value Provided' and Mandatory == False):
                            self.out_MG[self.out_MG.columns[column_count - 6]].values[i] = 'Error'
                            self.List[i] = self.List[i] +'The ' +reportlabel_leafattributename+ ' expected datatype is '+ data_type + ', '
                
                elif(data_type == 'integer'):
                    df_datacheck = self.out_MG.copy()
                    df_datacheck['check'] = df_datacheck[Attribute]
                    df_datacheck['Is_Integer'] = list(map(lambda x: x.isdigit(), df_datacheck[Attribute]))
                    #df_datacheck = self.out_MG[Attribute].apply(lambda x: x.str.isnumeric())
                    logger.info('{}'.format(df_datacheck.to_string()))
                    #mask = (df_datacheck['Is_Integer'] == False)
                    #self.out_MG[self.out_MG.columns[column_count - 6]][mask] = 'Error'
                    index_list = df_datacheck.index[df_datacheck['Is_Integer'] == False].tolist()
                    logger.info(index_list)
                    for i in index_list:
                        if(df_datacheck['check'].values[i] != None and Mandatory):
                            self.out_MG[self.out_MG.columns[column_count - 6]].values[i] = 'Error'
                            self.List[i] = self.List[i] +'The ' +reportlabel_leafattributename+ ' expected datatype is '+ data_type + ', '
                        elif(df_datacheck['check'].values[i] != 'No Value Provided' and Mandatory == False):
                            self.out_MG[self.out_MG.columns[column_count - 6]].values[i] = 'Error'
                            self.List[i] = self.List[i] +'The ' +reportlabel_leafattributename+ ' expected datatype is '+ data_type + ', '
                    logger.info('{}'.format(self.out_MG.to_string()))
                elif(data_type == 'datetime'):
                    df_datacheck2 = self.out_MG.copy()
                    df_datacheck2['check'] = df_datacheck2[Attribute]
                    df_datacheck2[Attribute] = pd.to_datetime(df_datacheck2[Attribute], errors='coerce')
                    df_datacheck2[Attribute] = df_datacheck2[Attribute].replace(pd.NaT,'False',regex = True)
                    #df_datacheck2[Attribute] = df_datacheck2[Attribute].apply(lambda x: x if isinstance(x,datetime) else 'False').fillna(method = "ffill")
                    logger.info('{}'.format(df_datacheck2.to_string()))
                    #df_datacheck2.to_csv('data_1.csv')
                    #mask = (df_datacheck2[Attribute] == 'False' and df_datacheck2['check'] not in None)
                    #self.out_MG[self.out_MG.columns[column_count - 6]][mask] = 'Error'
                    index_list = df_datacheck2.index[df_datacheck2[Attribute] == 'False'].tolist()
                    for i in index_list:
                        if(df_datacheck2['check'].values[i] != None and Mandatory):
                            self.out_MG[self.out_MG.columns[column_count - 6]].values[i] = 'Error'
                            self.List[i] = self.List[i] +'The ' +reportlabel_leafattributename+ ' expected datatype is '+ data_type + ', '
                        elif(df_datacheck2['check'].values[i] != 'No Value Provided' and Mandatory == False):
                            self.out_MG[self.out_MG.columns[column_count - 6]].values[i] = 'Error'
                            self.List[i] = self.List[i] +'The ' +reportlabel_leafattributename+ ' expected datatype is '+ data_type + ', '
    
    
            logger.info('{}'.format(self.out_MG.to_string()))
            logger.info("---------------Data_Typecheck end-----------------------------")        
    
    def Duplicate_datacheck(self):
        logger.info("---------------Duplicate Start-----------------------------")
        logger.info("---------------Data to this Function-----------------------------")
        logger.info('{}'.format(self.out_MG.to_string()))
        logger.info('{}'.format(self.mapping.to_string()))
        reportlabel_leafattributename =self.mapping.loc[self.mapping['Dim Attribute.[Dim Attribute Name]'] == self.leafattributename, 'Report Label'].item()
        
        Duplicate_df = pd.concat([ self.dim_data,self.user_input])
        Duplicate_df = Duplicate_df.drop_duplicates(subset=['Unique Identifier.[Unique Identifier]', self.leafattributename],keep='last').reset_index(drop=True)
        duplicate_rows = Duplicate_df[Duplicate_df.duplicated(self.leafattributename)]
        
        filter = duplicate_rows['Unique Identifier.[Unique Identifier]']
        column_count = len(self.out_MG.columns)
        for x in filter:
            mask = (self.out_MG['Unique Identifier.[Unique Identifier]'] == x)
            self.out_MG[self.out_MG.columns[column_count - 6]][mask] = 'Error'
            index_list = self.out_MG.index[self.out_MG['Unique Identifier.[Unique Identifier]'] == x].tolist()
            for i in index_list:
                self.List[i] = self.List[i] +'The ' +reportlabel_leafattributename+ ' attribute already exists in the Master '+ ', '
            
        logger.info("---------------Result Set of Data Duplicate Check-----------------------------")
        logger.info('{}'.format(self.out_MG.to_string()))
        logger.info(self.List)
        logger.info("---------------Duplicate End-----------------------------")
    
    def Hierarchy_check(self):
        logger.info("---------------Hierarchy Start-----------------------------")
        logger.info("---------------Data to this Function-----------------------------")
        logger.info('{}'.format(self.hierarchy.to_string()))
        logger.info('{}'.format(self.mapping.to_string()))
        
        self.hierarchy[['Hierarchy_Name', 'Attribute']] = self.hierarchy['_SchemaDimension.[_Level]'].str.split('.',expand=True)
        self.hierarchy = self.hierarchy.groupby('Hierarchy_Name').apply(lambda x: x.sort_values(['_SchemaDimension.[LevelPosition]'], ascending=True)).reset_index(drop=True)
        column_names = ["Hierarchy_Name", "LSParentColumnName", "LSChildColumnName"]
        Hierrarchy_Info = pd.DataFrame(columns=column_names)
        count = len(self.hierarchy.index)
        column_count = len(self.hierarchy.columns)
        ls_dimensionname = self.leafattributename.split('.')
        logger.info(ls_dimensionname)
        for j in range(count - 1):
            Input1 = ls_dimensionname[0]+'.' + self.hierarchy.iloc[j, column_count - 1]
            Input2 = self.hierarchy.iloc[j, column_count - 2]
            Input3 = ls_dimensionname[0]+'.' + self.hierarchy.iloc[j + 1, column_count - 1]
            Input4 = self.hierarchy.iloc[j + 1, column_count - 2]
            if (Input2 == Input4 and "All" not in Input1):
                Hierrarchy_Info.loc[len(Hierrarchy_Info.index)] = [Input2, Input1, Input3]
            else:
                continue
        
        #Hierrarchy_Info.to_excel("self_Hierrarchy_Info.xlsx")
        logger.info(Hierrarchy_Info)
        column_count1 = len(self.out_MG.columns)
        out_MG_valid = self.out_MG[(self.out_MG[self.out_MG.columns[column_count1 - 6]] != 'Error')]
        # out_MG_valid.to_excel("self_out_MG_valid.xlsx")
        for l in range(self.Total_attributes):
            index_no = self.mapping.columns.get_loc('Dim Attribute.[Dim Attribute Name]')
            oldName = self.mapping.iloc[l, index_no + 1]
            #### Updated on 18th feb 2022 start
            oldName = 'OUT ' + oldName
            #print(oldName)
            # logger.info(oldName)
            #### Updated on 18th feb 2022 end
            newName = self.mapping.iloc[l, index_no]
            out_MG_valid.rename(columns={oldName: newName}, inplace=True)
        
        # union = pd.concat([df1, df2], ignore_index=True)
    
        dimdata_valid = self.dim_data.copy()
        dimdata_valid= dimdata_valid.astype(str)
        out_MG_valid = out_MG_valid.astype(str)
        filter = dimdata_valid['Unique Identifier.[Unique Identifier]']
        for x in filter:
            if x in out_MG_valid['Unique Identifier.[Unique Identifier]'].values:
                dimdata_valid.drop(dimdata_valid[dimdata_valid['Unique Identifier.[Unique Identifier]'] == x].index,inplace=True)
        Combine_data = pd.concat([dimdata_valid,out_MG_valid], ignore_index=True, sort=False)
        #Combine_data.to_excel("self_Combine_data.xlsx")
        row_count = len(Hierrarchy_Info.index)
        column_count2 = len(self.out_MG.columns)
        for m in range(row_count):
            Child_Column = Hierrarchy_Info['LSChildColumnName'].values[m]
            Parent_Column = Hierrarchy_Info['LSParentColumnName'].values[m]
            temp_df = Combine_data.groupby([Child_Column, Parent_Column]).size().reset_index(name='counts')
            
            duplicate_row1 = temp_df[temp_df.duplicated(Child_Column)]
               
            row_count1 = len(temp_df.index)
            duplicate_row1 = duplicate_row1.groupby([Child_Column]).size().reset_index(name='counts')
            # output_hierarchy = temp_df[Child_Column]
            filter_values = duplicate_row1[Child_Column]
                
            ChildColumn_ReportLabel =self.mapping.loc[self.mapping['Dim Attribute.[Dim Attribute Name]'] == Child_Column, 'Report Label'].item()
            out_Child_Column = self.mapping.loc[self.mapping['Dim Attribute.[Dim Attribute Name]'] == Child_Column, 'Mapping Attribute'].item()
            out_Child_Column =  'OUT ' + out_Child_Column
         
                
            ParentColumn_ReportLabel =self.mapping.loc[self.mapping['Dim Attribute.[Dim Attribute Name]'] == Parent_Column, 'Report Label'].item()
            out_Parent_Column = self.mapping.loc[self.mapping['Dim Attribute.[Dim Attribute Name]'] == Parent_Column, 'Mapping Attribute'].item()
            out_Parent_Column =  'OUT ' + out_Parent_Column
            
            

            i_value = self.out_MG.first_valid_index()
            

            for x in filter_values:
                index_list = self.out_MG.index[self.out_MG[out_Child_Column] == x].tolist()
                count = len(index_list)
                
                check = 0
                check1 = 0
                for i in index_list:
                    if (self.out_MG['Unique Identifier.[Unique Identifier]'].values[i-i_value] not in out_MG_valid['Unique Identifier.[Unique Identifier]'].values):
                        continue
                    Child_value = self.out_MG[out_Child_Column].values[i-i_value]
                    Parent_value = self.out_MG[out_Parent_Column].values[i-i_value]
                    if (count == 1):
                        self.out_MG[self.out_MG.columns[column_count2 - 6]].values[i-i_value] = 'Error'
                        self.List[i] = self.List[i] + ChildColumn_ReportLabel +' ' + Child_value + ' has Multiple Parents:  current ' + ParentColumn_ReportLabel + ' value is ' + Parent_value + ' ' + ', '
                    elif(count > 1):
                        if(Child_value in self.dim_data[Child_Column].values and Parent_value in self.dim_data[Parent_Column].values):
                            check = 1
                        elif(Child_value in self.dim_data[Child_Column].values and Parent_value not in self.dim_data[Parent_Column].values):
                            check = 1
                            self.out_MG[self.out_MG.columns[column_count2 - 6]].values[i-i_value] = 'Error'
                            self.List[i] = self.List[i] + ChildColumn_ReportLabel +' ' + Child_value + ' has Multiple Parents:  current ' + ParentColumn_ReportLabel + ' value is ' + Parent_value + ' ' + ', '
                    if(count > 1 and check ==0):
                        if(check1 != 0):
                            self.out_MG[self.out_MG.columns[column_count2 - 6]].values[i-i_value] = 'Error'
                            self.List[i] = self.List[i] + ChildColumn_ReportLabel +' ' + Child_value + ' has Multiple Parents:  current ' + ParentColumn_ReportLabel + ' value is ' + Parent_value + ' ' + ', '
                        check1 = check1 + 1
        logger.info("---------------Result Set of Hierarchy Check-----------------------------")
        logger.info('{}'.format(self.out_MG.to_string()))
        logger.info(self.List)
        logger.info("---------------Hierarchy End-----------------------------")
    
    def Generate_LogInfo(self):
        logger.info("---------------Log Start-----------------------------")
        logger.info("---------------Data to this Function-----------------------------")
        logger.info('{}'.format(self.out_MG.to_string()))
        logger.info('{}'.format(self.mapping.to_string()))
        #logger.info('{}'.format(self.Total_attributes.to_string()))
        
        column_names = ["Version Name", "Transaction ID", "Master Data Name", "Unique Identifier", "LOG Data", "LOG Status", "LOG ModifiedBy", "LOG Modified On"]
        
        Log_Info_df = pd.DataFrame(columns=column_names)
        column_count = len(self.out_MG.index)
        self.out_MG.fillna('', inplace=True)
        out_MGLog_df = self.out_MG.copy()
        out_MGLog_df["Data"] = 'datainfo'
        
        numberofcolumnsinOUTMG = len(out_MGLog_df.columns)
    
        for l in range(self.Total_attributes):
            index_no = self.mapping.columns.get_loc('Dim Attribute.[Dim Attribute Name]')
            oldName = self.mapping.iloc[l, index_no + 1]
            #### Updated on 18th feb 2022 start
            oldName = 'OUT ' + oldName
            #print(oldName)
            #logger.info(oldName)
            #### Updated on 18th feb 2022 end
            out_MGLog_df['Data'] = out_MGLog_df.apply(lambda x: '%s|~|%s' % (x['Data'], x[oldName]), axis=1)
            
        for p in range(column_count):
            Log_Info_df.loc[len(Log_Info_df.index)] = [
                out_MGLog_df['Version.[Version Name]'].values[p],
                self.transaction_id,
                out_MGLog_df['Master Data.[Master Data Name]'].values[p],
                out_MGLog_df['Unique Identifier.[Unique Identifier]'].values[p],
                str(out_MGLog_df['Data'].values[p]).replace("datainfo|~|","").strip(),
                out_MGLog_df[out_MGLog_df.columns[numberofcolumnsinOUTMG - 7]].values[p],
                out_MGLog_df[out_MGLog_df.columns[numberofcolumnsinOUTMG - 6]].values[p],
                out_MGLog_df[out_MGLog_df.columns[numberofcolumnsinOUTMG - 5]].values[p]
                ]
        #Log_Info_df.to_excel("MG_LOG.xlsx")
        logger.info("---------------Result Set of LOG--------------------------")
        logger.info('{}'.format(Log_Info_df.to_string()))
        Log_Info_df = Log_Info_df.astype(str)
        logger.info('{}'.format(Log_Info_df.to_string()))
        logger.info("---------------Log End---------------------------")
        return Log_Info_df
    
    def Update_InputandOutputMG(self):
        logger.info("---------------Input Update Status Start---------------------------")
        logger.info("---------------Data to this Function-----------------------------")
        logger.info('{}'.format(self.out_MG.to_string()))
        #self.out_MG.fillna('', inplace=True)
        
        column_count = len(self.out_MG.columns)
        OutMeasureName = 'OUT '+self.leafattributename.split('.')[0] + ' LSKey'
        
        #row_count = len(self.out_MG.index)
        
        #for m in range(row_count):
            #key_value = self.out_MG['Unique Identifier.[Unique Identifier]'].values[m]
            #logger.info(self.out_MG[self.out_MG.columns[column_count - 6]])
            #if (key_value not in self.dim_data['Unique Identifier.[Unique Identifier]'].values):
                #self.out_MG[self.out_MG.columns[column_count - 1]].values[m] = self.current_time.strftime("%Y-%m-%d %H:%M:%S")
                #self.out_MG[self.out_MG.columns[column_count - 2]].values[m] = self.username
            
            
            
     
        
        
        logger.info(self.out_MG[self.out_MG.columns[column_count - 6]])
        mask = (self.out_MG[self.out_MG.columns[column_count - 6]] == "Added")
        self.out_MG[self.out_MG.columns[column_count - 1]][mask] = self.current_time.strftime("%Y-%m-%d %H:%M:%S")
        self.out_MG[self.out_MG.columns[column_count - 2]][mask] = self.username
    
        
        self.out_MG[OutMeasureName] = self.out_MG['Unique Identifier.[Unique Identifier]'].astype(int)
        
        #for l in range(self.Total_attributes):
            #index_no = self.mapping.columns.get_loc('Dim Attribute.[Dim Attribute Name]')
            #oldName = self.mapping.iloc[l, index_no + 1]
            #oldName = 'OUT ' + oldName
            #self.out_MG[oldName].fillna('', inplace=True)
            #self.out_MG[oldName] = self.out_MG[oldName].astype(str)
        
        
        
        
        logger.info('{}'.format(self.out_MG.to_string()))
        
        logger.info('####')
        #self.out_MG[self.out_MG.columns[column_count - 1]] = self.out_MG[self.out_MG.columns[column_count - 1]].fillna('NaT', inplace=True)
        self.out_MG[self.out_MG.columns[column_count - 1]] = self.out_MG[self.out_MG.columns[column_count - 1]].replace(r'^\s*$', '', regex=True)
        
        self.out_MG= self.out_MG.astype(str)
        logger.info('{}'.format(self.out_MG.to_string()))
        logger.info('####')
        
        Column_Name = self.out_MG.columns[column_count - 3]

        self.out_MG[Column_Name] = 'raj'

        for i in self.out_MG.index:
            self.out_MG[Column_Name].values[i] = self.List[i]
    
        self.out_MG[Column_Name] = self.out_MG[Column_Name].str.replace("List", "")
        #self.out_MG.to_excel("MG_Output.xlsx")
        out_MGInput_df = self.out_MG.copy()
        
        for l in range(self.Total_attributes):
            index_no = self.mapping.columns.get_loc('Dim Attribute.[Dim Attribute Name]')
            Mandatory = self.mapping['Is Mandatory'].values[l]
            oldName = self.mapping.iloc[l, index_no + 1]
            oldName = 'OUT ' + oldName
            if(Mandatory):
                out_MGInput_df = out_MGInput_df.drop([oldName], axis = 1)
            
        
        out_MGInput_df.columns = out_MGInput_df.columns.str.replace("OUT", "IN")
        #out_MGInput_df.to_excel("MG_InputStatus.xlsx")
            
        logger.info('{}'.format(out_MGInput_df.to_string()))
        logger.info("---------------Input Update Status End---------------------------")
        return out_MGInput_df,self.out_MG
    
    def main(self):
        # main function should primarily be used for calling other functions
        logger.info('---------Start The Function Call-----------------------------')
        self.Rename_Column()
        self.Netchange()
        self.Mandatory_datacheck()
        self.Data_Typecheck()
        self.Duplicate_datacheck()
        self.Hierarchy_check()
        a_finalmg_loginfo = self.Generate_LogInfo()
        a_finalmg_inputinfo,a_finalmg_outputinfo = self.Update_InputandOutputMG()
        logger.info('---------END The MAIN Function Call-----------------------------')
        
        a_finalmg_inputinfo.rename(columns={'Master Data.[Master Data Name]': '[Master Data].[Master Data Name]'}, inplace=True)
        a_finalmg_inputinfo.rename(columns={'Unique Identifier.[Unique Identifier]': '[Unique Identifier].[Unique Identifier]'}, inplace=True)
            
        a_finalmg_outputinfo.rename(columns={'Master Data.[Master Data Name]': '[Master Data].[Master Data Name]'}, inplace=True)
        a_finalmg_outputinfo.rename(columns={'Unique Identifier.[Unique Identifier]': '[Unique Identifier].[Unique Identifier]'}, inplace=True)

        a_finalmg_loginfo.rename(columns={'Version Name': 'Version.[Version Name]'}, inplace=True)
        a_finalmg_loginfo.rename(columns={'Transaction ID': '[Transaction ID].[Transaction ID]'}, inplace=True)
        a_finalmg_loginfo.rename(columns={'Master Data Name': '[Master Data].[Master Data Name]'}, inplace=True)
        a_finalmg_loginfo.rename(columns={'Unique Identifier': '[Unique Identifier].[Unique Identifier]'}, inplace=True)

        logger.info('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
        logger.info('-------------------------FINAL RESULTS-----------------------')
        logger.info('{}'.format(a_finalmg_inputinfo.to_string()))
        logger.info('{}'.format(a_finalmg_outputinfo.to_string()))
        logger.info('{}'.format(a_finalmg_loginfo.to_string()))
        logger.info('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
            
        finalmg_loginfo = a_finalmg_loginfo
        finalmg_inputinfo = a_finalmg_inputinfo
        finalmg_outputinfo = a_finalmg_outputinfo
            
        logger.info('{}'.format(finalmg_inputinfo.to_string()))
        logger.info('{}'.format(finalmg_outputinfo.to_string()))
        logger.info('{}'.format(finalmg_loginfo.to_string()))
        logger.info('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')            
            
            
        logger.info('---------MDM Commit End-----------------------------')
        return finalmg_loginfo, finalmg_inputinfo, finalmg_outputinfo
    
MDM = MasterData()
#MDM.main()
finalmg_loginfo, finalmg_inputinfo, finalmg_outputinfo = MDM.main()
